# Word Ladder

## Version 1 - return steps

Given two words (beginWord and endWord), and a dictionary's word list, find the
length of shortest transformation sequence from beginWord to endWord, such that:

  - Only *one letter* can be changed at a time.
  - Each transformed word must exist in the word list.

Note:
  - Return 0 if there is no such transformation sequence.
  - All words have the same length.
  - All words contain only lowercase alphabetic characters.
  - You may assume no duplicates in the word list.
  - You may assume beginWord and endWord are non-empty and are not the same.

EXAMPLES
```
Input: beginWord = "hit", endWord = "cog",
  wordList = ["hot","dot","dog","lot","log","cog"]
Output: 5
Explanation: "hit" -> "hot" -> "dot" -> "dog" -> "cog" (length = 5)

Input: beginWord = "hit", endWord = "cog", 
  wordList = ["hot","dot","dog","lot","log"]
Output: 0
Explanation: The endWord "cog" is not in wordList, therefore no possible transformation.
```

Ref:
  - https://leetcode.com/problems/word-ladder (Hard)
  - https://www.geeksforgeeks.org/word-ladder-length-of-shortest-chain-to-reach-a-target-word/

In [23]:
from typing import List
from collections import defaultdict


class Solution:

    def ladderLength_v1(self, beginWord: str, endWord: str, wordList: List[str]) -> int:
        """Perform a Bread-First Search (BFS) to guarantee the optimal solution.

        Complexity is O(n^2 m), where n is the number of words in the dictionary
        and m is the length of the word.
        For the worst case, each word in the dictionary needs to check with each other.
        Thus, O(n^2).  The cost to get the distance of two words is m.
        """
        def word_distance(w1, w2):
            """Return the distance of two words (of the same length)."""
            n = 0
            for c1, c2 in zip(w1, w2):
                if c1 != c2:
                    n += 1
            return n

        if beginWord == endWord:
            return 1

        queue = [beginWord]
        visited = set([beginWord])
        length = 2
        while queue:
            n = len(queue)
            for i in range(n):
                curr = queue.pop(0)
                for w in wordList:
                    if w in visited:
                        continue
                    if word_distance(curr, w) == 1:
                        if w == endWord:
                            return length
                        else:
                            queue.append(w)
                            visited.add(w)
            length += 1

        return 0

    def ladderLength_v2(self, beginWord: str, endWord: str, wordList: List[str]) -> int:
        """Use word patterns to speed up distance comparison.
        E.g., 
            h*t -> [hit, hat, hot, ...]
            do* -> [dog, dot, ...]
        """

        # Sanity check
        if endWord not in wordList:
            return 0

        # Pre-process all words and built indices
        graph = defaultdict(list)   # patterns back to words
        for word in wordList:
            for i in range(len(word)):
                pattern = word[0:i] + '*' + word[i+1:]
                graph[pattern].append(word)

        # Begin to search
        queue = [beginWord]
        visited = set()
        length = 1
        while queue:
            tmp_queue = set()
            for word in queue:
                visited.add(word)
                for i in range(len(word)):
                    pattern = word[0:i] + '*' + word[i+1:]
                    for node in graph[pattern]:
                        if node == endWord:
                            return length + 1
                        elif node in visited:
                            continue
                        tmp_queue.add(node)
            queue = tmp_queue
            length += 1
        return 0

    def ladderLength_v3(self, beginWord: str, endWord: str, wordList: List[str]) -> int:
        """Two-side BFS -- search from both begin and end.

        Use two queues -- queue0 and queue1. 
        Then either use a flag to indicate the direction or swap these two queues.
        """

        # Sanity check
        if endWord not in wordList:
            return 0

        graph = defaultdict(list)   # patterns back to words
        for word in wordList:
            for i in range(len(word)):
                pattern = word[0:i] + '*' + word[i+1:]
                graph[pattern].append(word)

        # Begin to search
        queue0 = [beginWord]
        queue1 = [endWord]
        visited = set([beginWord, endWord])
        length = 1

        while queue0 and queue1:
            tmp_queue = set()
            for word in queue0:
                visited.add(word)
                for i in range(len(word)):
                    pattern = word[0:i] + '*' + word[i+1:]
                    for node in graph[pattern]:
                        if node in queue1:
                            return length + 1
                        if node in visited:
                            continue
                        tmp_queue.add(node)
            length += 1
            queue0 = tmp_queue

            # Swap queues!
            queue0, queue1 = queue1, queue0

        return 0


def main():
    test_data = [
        ['hit', 'cog', ['hot', 'dot', 'dog', 'lot', 'log', 'cog'], 5],
        ['hit', 'cog', ['hot', 'dot', 'dog', 'lot', 'log'], 0],
        ["red", "tax", ["ted", "tex", "red", "tax", "tad", "den", "rex", "pee"], 4],
        ['qa', 'sq', ["si", "go", "se", "cm", "so", "ph", "mt", "db", "mb", "sb", "kr", "ln",
         "tm", "le", "av", "sm", "ar", "ci", "ca", "br", "ti", "ba", "to", "ra", "fa", "yo",
         "ow", "sn", "ya", "cr", "po", "fe", "ho", "ma", "re", "or", "rn", "au", "ur", "rh",
         "sr", "tc", "lt", "lo", "as", "fr", "nb", "yb", "if", "pb", "ge", "th", "pm", "rb",
         "sh", "co", "ga", "li", "ha", "hz", "no", "bi", "di", "hi", "qa", "pi", "os", "uh",
         "wm", "an", "me", "mo", "na", "la", "st", "er", "sc", "ne", "mn", "mi", "am", "ex",
         "pt", "io", "be", "fm", "ta", "tb", "ni", "mr", "pa", "he", "lr", "sq", "ye"], 5],
    ]

    ob1 = Solution()
    for beginWord, endWord, wordList, ans in test_data:
        print(f"# Input: begin={beginWord}, end={endWord} (ans={ans})")
        print(f"  - words={wordList}")
        print(f"  Output v1 = {ob1.ladderLength_v1(beginWord, endWord, wordList)}")
        print(f"  Output v2 = {ob1.ladderLength_v2(beginWord, endWord, wordList)}")
        print(f"  Output v3 = {ob1.ladderLength_v3(beginWord, endWord, wordList)}")


main()

# Input: begin=hit, end=cog (ans=5)
  - words=['hot', 'dot', 'dog', 'lot', 'log', 'cog']
  Output v1 = 5
  Output v2 = 5
  Output v3 = 5
# Input: begin=hit, end=cog (ans=0)
  - words=['hot', 'dot', 'dog', 'lot', 'log']
  Output v1 = 0
  Output v2 = 0
  Output v3 = 0
# Input: begin=red, end=tax (ans=4)
  - words=['ted', 'tex', 'red', 'tax', 'tad', 'den', 'rex', 'pee']
  Output v1 = 4
  Output v2 = 4
  Output v3 = 4
# Input: begin=qa, end=sq (ans=5)
  - words=['si', 'go', 'se', 'cm', 'so', 'ph', 'mt', 'db', 'mb', 'sb', 'kr', 'ln', 'tm', 'le', 'av', 'sm', 'ar', 'ci', 'ca', 'br', 'ti', 'ba', 'to', 'ra', 'fa', 'yo', 'ow', 'sn', 'ya', 'cr', 'po', 'fe', 'ho', 'ma', 're', 'or', 'rn', 'au', 'ur', 'rh', 'sr', 'tc', 'lt', 'lo', 'as', 'fr', 'nb', 'yb', 'if', 'pb', 'ge', 'th', 'pm', 'rb', 'sh', 'co', 'ga', 'li', 'ha', 'hz', 'no', 'bi', 'di', 'hi', 'qa', 'pi', 'os', 'uh', 'wm', 'an', 'me', 'mo', 'na', 'la', 'st', 'er', 'sc', 'ne', 'mn', 'mi', 'am', 'ex', 'pt', 'io', 'be', 'fm', 'ta', 'tb', 'ni', 'mr

---
## Version 2 - return word sequence

Given two words (beginWord and endWord), and a dictionary's word list, find *ALL*
shortest transformation sequence(s) from beginWord to endWord, such that:

  - Only *one letter* can be changed at a time.
  - Each transformed word must exist in the word list.

Note:
  - Return an empty list if there is no such transformation sequence.
  - All words have the same length.
  - All words contain only lowercase alphabetic characters.
  - You may assume no duplicates in the word list.
  - You may assume beginWord and endWord are non-empty and are not the same.

EXAMPLES
```
Input: 
    beginWord = "hit",
    endWord = "cog",
    wordList = ["hot","dot","dog","lot","log","cog"]
Output:
    [
      ["hit","hot","dot","dog","cog"],
      ["hit","hot","lot","log","cog"]
    ]

Input: 
    beginWord = "hit",
    endWord = "cog",
    wordList = ["hot","dot","dog","lot","log"]
Output:
    []
Explanation: The endWord "cog" is not in wordList.
```

HINTS:
  - Need to build a reverse index to speed up matching.
  - Use bread-first search.
  - There may be multiple way to reach a node. Thus, need to track all of these paths.

Ref:
  - https://leetcode.com/problems/word-ladder-ii/ (Hard)
  - https://www.geeksforgeeks.org/word-ladder-length-of-shortest-chain-to-reach-a-target-word/


In [3]:
from typing import List
from collections import defaultdict


class Solution:

    def findLadders_v1(self, beginWord: str, endWord: str, wordList: List[str]) -> List[List[str]]:
        """Naive BFS."""
        if endWord not in wordList:
            return []

        # Create the pattern to words mapping
        graph = defaultdict(list)
        for word in wordList:
            for i in range(len(word)):
                graph[word[:i] + "*" + word[i+1:]].append(word)

        ans = []
        queue = {beginWord: [[beginWord]]}
        visited = {beginWord}
        while queue and not ans:
            temp_queue = defaultdict(list)
            for word, paths in queue.items():
                for i in range(len(word)):
                    pattern = word[:i] + "*" + word[i+1:]
                    for node in graph.get(word[:i] + "*" + word[i+1:], []):
                        if node == endWord:
                            for path in paths:
                                ans.append(path + [node])
                        if node in visited:
                            continue
                        # Note that there can be more than one way to reach to this node.
                        # Thus, paths is a list of lists (sequences)
                        for path in paths:
                            temp_queue[node].append(path + [node])
            # Has to be updated level-by-level
            visited |= set(temp_queue.keys())
            queue = temp_queue
        return ans

    def findLadders_v2(self, beginWord: str, endWord: str, wordList: List[str]) -> List[List[str]]:
        """Two-end BFS."""
        if endWord not in wordList:
            return []

        # Build index
        graph = dict()
        for word in wordList:
            for i in range(len(word)):
                graph.setdefault(word[:i] + "*" + word[i+1:], []).append(word)

        ans = []
        queue0 = {beginWord: [[beginWord]]}
        queue1 = {endWord: [[endWord]]}
        visited = {beginWord, endWord}
        reverse = False

        while queue0 and queue1 and not ans:
            temp_queue = defaultdict(list)
            for word, seq in queue0.items():
                for i in range(len(word)):
                    for node in graph.get(word[:i] + "*" + word[i+1:], []):
                        # Front and end queues intersept
                        if node in queue1:
                            for x in seq:
                                for y in queue1[node]:
                                    path = y + \
                                        x[::-1] if reverse else x + y[::-1]
                                    ans.append(path)

                        if node in visited:
                            continue

                        # Append node to the seq.
                        # Here we don't need to worry if it is reversed or not.
                        for x in seq:
                            temp_queue[node].append(x + [node])

            # has to be updated level-by-level
            visited |= set(temp_queue.keys())
            queue0 = temp_queue

            # Swap front queue and rear queue
            queue0, queue1 = queue1, queue0
            reverse = not reverse

        return ans


def main():
    test_data = [
        ['hit', 'cog', ['hot', 'dot', 'dog', 'lot', 'log', 'cog']],  # 2
        ['hit', 'cog', ['hot', 'dot', 'dog', 'lot', 'log']],  # []
        ["red", "tax", ["ted", "tex", "red", "tax", "tad", "den", "rex", "pee"]],  # 3
    ]

    sol = Solution()
    for beginWord, endWord, wordList in test_data:
        print("# Input: {}, {}, {}".format(beginWord, endWord, wordList))
        print("  Output v1 = {}".format(
            sol.findLadders_v1(beginWord, endWord, wordList)))
        print("  Output v2 = {}".format(
            sol.findLadders_v2(beginWord, endWord, wordList)))


if __name__ == "__main__":
    main()


# Input: hit, cog, ['hot', 'dot', 'dog', 'lot', 'log', 'cog']
  Output v1 = [['hit', 'hot', 'dot', 'dog', 'cog'], ['hit', 'hot', 'lot', 'log', 'cog']]
  Output v2 = [['hit', 'hot', 'dot', 'dog', 'cog'], ['hit', 'hot', 'lot', 'log', 'cog']]
# Input: hit, cog, ['hot', 'dot', 'dog', 'lot', 'log']
  Output v1 = []
  Output v2 = []
# Input: red, tax, ['ted', 'tex', 'red', 'tax', 'tad', 'den', 'rex', 'pee']
  Output v1 = [['red', 'ted', 'tad', 'tax'], ['red', 'ted', 'tex', 'tax'], ['red', 'rex', 'tex', 'tax']]
  Output v2 = [['red', 'ted', 'tad', 'tax'], ['red', 'ted', 'tex', 'tax'], ['red', 'rex', 'tex', 'tax']]
